In [1]:
#SMC Meta Challange

In [2]:
import numpy as np
import pandas as pd
from collections import Counter

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score
from sklearn import metrics
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.grid_search import GridSearchCV
from sklearn import svm
from sklearn.decomposition import RandomizedPCA
from sklearn.metrics import classification_report
from sklearn import preprocessing
import matplotlib.pyplot as plt
import scipy.optimize as opt
import glob
import os
import sys
import pickle
import time
import timeit

from sklearn.decomposition import RandomizedPCA

import theano
import theano.tensor as T
import lasagne

#np.random.seed(1337)  # for reproducibility
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten, Layer, Reshape, AutoEncoder, Merge
from keras.layers.convolutional import Convolution2D,MaxPooling2D,Convolution1D,MaxPooling1D,UpSampling2D, ZeroPadding2D
from keras.optimizers import SGD, Adam, RMSprop, Adadelta
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
import keras.utils.visualize_util as vutil
from keras.layers.recurrent import LSTM, GRU, SimpleRNN
from keras.regularizers import l1, l2, l1l2, activity_l2, activity_l1
import keras.layers.containers as containers
from keras.layers.noise import GaussianNoise
from keras.regularizers import ActivityRegularizer
from keras import backend as K
import keras.utils.visualize_util as vutil
from keras.models import model_from_json

import seaborn as sns

%matplotlib inline

from IPython.display import SVG

import h5py

np.set_printoptions(suppress=True)  #Supress exponent of the number 2e-2 == 0.02

%matplotlib inline

WARNING (theano.sandbox.cuda): CUDA is installed, but device gpu is not available  (error: Unable to get the number of gpus available: no CUDA-capable device is detected)


Using Theano backend.


/usr/local/lib/python2.7/dist-packages/theano/tensor/signal/downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


In [3]:
#Read the data

GenomicFeatures_CPCG0100 = pd.read_csv(open('data/CPCG0100/GenomicFeatures_CPCG0100.txt','r'),sep='\t')
SNVCalls_CPCG0100    = pd.read_csv(open('data/CPCG0100/SNVCalls_CPCG0100.txt','r'),sep='\t') 

In [4]:
print GenomicFeatures_CPCG0100.shape
print SNVCalls_CPCG0100.shape

(198526, 15)
(198526, 182)


In [5]:
GenomicFeatures_CPCG0100.columns

Index([u'CHROM', u'POS', u'RefAllele', u'NonRefAllele', u'BaseQual', u'TumourCoverage', u'NormalCoverage', u'MapQual', u'ReadPosition', u'Trinucleotide', u'HomopolymerRate', u'GCcontent', u'GenomicLocation', u'DistanceSNP', u'StrandBias'], dtype='object')

In [6]:
GenomicFeatures_CPCG0100.head

<bound method DataFrame.head of    CHROM     POS  RefAllele  NonRefAllele  BaseQual  TumourCoverage  \
0   chr1   10408        183             3      4490             231   
1   chr1   10413        177            10      4691             229   
2   chr1   16580        115            10      3512             137   
3   chr1   17519         83            26      3251             119   
4   chr1   17538         96            20      3255             127   
5   chr1   20227        120            11      3672             155   
6   chr1   20250        117            14      3865             155   
7   chr1   28511         92             8      2946             123   
8   chr1   28593         27             3       524              36   
9   chr1   63171          9             3       342              17   
10  chr1   66394          7             0       211               7   
11  chr1   95740         53            17      1956              75   
12  chr1   98749         25            12    

In [7]:
SNVCalls_CPCG0100.columns

Index([u'CHROM', u'POS', u'END', u'X2399002', u'X2673208', u'X2673225', u'X2677201', u'X2677205', u'X2679037', u'X2679040', u'X2711267', u'X2762064', u'X2762066', u'X2787011', u'X2787148', u'X2787658', u'X2787662', u'X2788241', u'X2788246', u'X2788251', u'X2788254', u'X2791036', u'X2791292', u'X2791296', u'X2791300', u'X2791303', u'X2791323', u'X2791327', u'X2791331', u'X2791334', u'X2811002', u'X2811005', u'X2811008', u'X2811011', u'X2811383', u'X2812900', u'X2813239', u'X2813269', u'X2813271', u'X2813349', u'X2813361', u'X2813365', u'X2813411', u'X2813787', u'X2813792', u'X2813795', u'X2814925', u'X2815053', u'X2815103', u'X2815408', u'X2815466', u'X2815531', u'X2815569', u'X2815911', u'X2815941', u'X2815952', u'X2815972', u'X2816014', u'X2816109', u'X2816112', u'X2816115', u'X2816137', u'X2816157', u'X2816183', u'X2816186', u'X2816193', u'X2816274', u'X2816276', u'X2816278', u'X2816280', u'X2816282', u'X2816284', u'X2816286', u'X2816288', u'X2816290', u'X2816292', u'X2816294', u'X28

In [8]:
SNVCalls_CPCG0100.head

<bound method DataFrame.head of    CHROM     POS     END  X2399002  X2673208  X2673225  X2677201  X2677205  \
0   chr1   10407   10408         0         0         0         0         0   
1   chr1   10412   10413         0         0         0         0         0   
2   chr1   16579   16580         0         0         0         0         0   
3   chr1   17518   17519         0         0         0         0         0   
4   chr1   17537   17538         0         0         0         0         0   
5   chr1   20226   20227         0         0         0         0         0   
6   chr1   20249   20250         0         0         0         0         0   
7   chr1   28510   28511         0         0         0         0         0   
8   chr1   28592   28593         0         0         0         0         0   
9   chr1   63170   63171         0         0         0         0         0   
10  chr1   66393   66394         0         0         0         0         0   
11  chr1   95739   95740        

In [9]:
np.unique(SNVCalls_CPCG0100['CHROM'])

array(['chr1', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15',
       'chr16', 'chr17', 'chr18', 'chr19', 'chr2', 'chr20', 'chr21',
       'chr22', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9',
       'chrX', 'chrY'], 
      dtype='|S5')

In [10]:
CPCG0100 = pd.merge(GenomicFeatures_CPCG0100, SNVCalls_CPCG0100,left_on=['CHROM', 'POS'], right_on=['CHROM','END'])

#CPCG0100 = pd.merge(GenomicFeatures_CPCG0100, SNVCalls_CPCG0100,left_on=['POS'], right_on=['END'])

In [11]:
CPCG0100.shape #182+15

(198526, 196)

In [12]:
#CPCG0100.head

In [13]:
CPCG0100.columns

Index([u'CHROM', u'POS_x', u'RefAllele', u'NonRefAllele', u'BaseQual', u'TumourCoverage', u'NormalCoverage', u'MapQual', u'ReadPosition', u'Trinucleotide', u'HomopolymerRate', u'GCcontent', u'GenomicLocation', u'DistanceSNP', u'StrandBias', u'POS_y', u'END', u'X2399002', u'X2673208', u'X2673225', u'X2677201', u'X2677205', u'X2679037', u'X2679040', u'X2711267', u'X2762064', u'X2762066', u'X2787011', u'X2787148', u'X2787658', u'X2787662', u'X2788241', u'X2788246', u'X2788251', u'X2788254', u'X2791036', u'X2791292', u'X2791296', u'X2791300', u'X2791303', u'X2791323', u'X2791327', u'X2791331', u'X2791334', u'X2811002', u'X2811005', u'X2811008', u'X2811011', u'X2811383', u'X2812900', u'X2813239', u'X2813269', u'X2813271', u'X2813349', u'X2813361', u'X2813365', u'X2813411', u'X2813787', u'X2813792', u'X2813795', u'X2814925', u'X2815053', u'X2815103', u'X2815408', u'X2815466', u'X2815531', u'X2815569', u'X2815911', u'X2815941', u'X2815952', u'X2815972', u'X2816014', u'X2816109', u'X2816112', 

In [14]:
CPCG0100_X = CPCG0100.iloc[:,2:15] #Exclude CHROM, POS
CPCG0100_Y = CPCG0100.iloc[:,17:]  #Exclude CHROM_y, POS_y, END

In [15]:
print CPCG0100_X.shape
print CPCG0100_X.columns

(198526, 13)
Index([u'RefAllele', u'NonRefAllele', u'BaseQual', u'TumourCoverage', u'NormalCoverage', u'MapQual', u'ReadPosition', u'Trinucleotide', u'HomopolymerRate', u'GCcontent', u'GenomicLocation', u'DistanceSNP', u'StrandBias'], dtype='object')


In [16]:
print CPCG0100_Y.shape
print CPCG0100_Y.columns
print CPCG0100_Y[:5]

(198526, 179)
Index([u'X2399002', u'X2673208', u'X2673225', u'X2677201', u'X2677205', u'X2679037', u'X2679040', u'X2711267', u'X2762064', u'X2762066', u'X2787011', u'X2787148', u'X2787658', u'X2787662', u'X2788241', u'X2788246', u'X2788251', u'X2788254', u'X2791036', u'X2791292', u'X2791296', u'X2791300', u'X2791303', u'X2791323', u'X2791327', u'X2791331', u'X2791334', u'X2811002', u'X2811005', u'X2811008', u'X2811011', u'X2811383', u'X2812900', u'X2813239', u'X2813269', u'X2813271', u'X2813349', u'X2813361', u'X2813365', u'X2813411', u'X2813787', u'X2813792', u'X2813795', u'X2814925', u'X2815053', u'X2815103', u'X2815408', u'X2815466', u'X2815531', u'X2815569', u'X2815911', u'X2815941', u'X2815952', u'X2815972', u'X2816014', u'X2816109', u'X2816112', u'X2816115', u'X2816137', u'X2816157', u'X2816183', u'X2816186', u'X2816193', u'X2816274', u'X2816276', u'X2816278', u'X2816280', u'X2816282', u'X2816284', u'X2816286', u'X2816288', u'X2816290', u'X2816292', u'X2816294', u'X2816296', u'X2

In [17]:
CPCG0100_X[:5]

,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias
0,183,3,4490,231,133,25,55,TTA,4.696078,0.587065,NaN,41830,0.367965
1,177,10,4691,229,134,25,60,CTA,4.528846,0.587065,NaN,41825,0.371179
2,115,10,3512,137,74,0,52,TCA,2.707143,0.532338,NaN,35658,0.335766
3,83,26,3251,119,59,37,38,GCA,2.746377,0.676617,ncRNA_intronic,34719,0.462185
4,96,20,3255,127,53,29,47,GCC,2.719424,0.676617,NaN,34700,0.409449


In [18]:
np.unique(CPCG0100_X['GenomicLocation'])   #Trinucleotide

array(['UTR3', 'UTR5', 'downstream', 'exonic', 'exonic;splicing',
       'intergenic', 'intronic', 'nan', 'ncRNA_UTR3', 'ncRNA_UTR5',
       'ncRNA_exonic', 'ncRNA_intronic', 'ncRNA_splicing', 'splicing',
       'upstream', 'upstream;downstream'], 
      dtype='|S19')

In [258]:
#CPCG0100_X['RefAllele'].isnull
pd.get_dummies(CPCG0100_X['GenomicLocation'])[:5]

,UTR3,UTR5,downstream,exonic,exonic;splicing,intergenic,intronic,ncRNA_UTR3,ncRNA_UTR5,ncRNA_exonic,ncRNA_intronic,ncRNA_splicing,splicing,upstream,upstream;downstream
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [259]:
np.unique(CPCG0100_X['GenomicLocation'])

array(['UTR3', 'UTR5', 'downstream', 'exonic', 'exonic;splicing',
       'intergenic', 'intronic', 'nan', 'ncRNA_UTR3', 'ncRNA_UTR5',
       'ncRNA_exonic', 'ncRNA_intronic', 'ncRNA_splicing', 'splicing',
       'upstream', 'upstream;downstream'], 
      dtype='|S19')

In [260]:
np.unique(CPCG0100_X['Trinucleotide'])

array(['ACA', 'ACC', 'ACG', 'ACT', 'ATA', 'ATC', 'ATG', 'ATT', 'CCA',
       'CCC', 'CCG', 'CCT', 'CNN', 'CTA', 'CTC', 'CTG', 'CTT', 'GCA',
       'GCC', 'GCG', 'GCT', 'GTA', 'GTC', 'GTG', 'GTT', 'NCG', 'NTC',
       'TCA', 'TCC', 'TCG', 'TCT', 'TTA', 'TTC', 'TTG', 'TTT'], 
      dtype='|S3')

In [261]:
Trinucleotide_onehot   = pd.get_dummies(CPCG0100_X['Trinucleotide'])
GenomicLocation_onehot = pd.get_dummies(CPCG0100_X['GenomicLocation'])

print Trinucleotide_onehot.shape
print GenomicLocation_onehot.shape

(198526, 35)
(198526, 15)


In [293]:
print CPCG0100_X.shape

CPCG0100_X1 = CPCG0100_X.drop(['Trinucleotide', 'GenomicLocation'],axis=1)

print CPCG0100_X1.shape

(198526, 13)
(198526, 11)


In [294]:
print Trinucleotide_onehot.isnull().sum().sum()  
print GenomicLocation_onehot.isnull().sum().sum()  

0
0


In [295]:
print CPCG0100_X1.columns
print CPCG0100_X1['StrandBias'].isnull().sum().sum()  #32+32 # ReadPosition + StrandBias
print CPCG0100_X1.isnull().sum().sum() 

Index([u'RefAllele', u'NonRefAllele', u'BaseQual', u'TumourCoverage', u'NormalCoverage', u'MapQual', u'ReadPosition', u'HomopolymerRate', u'GCcontent', u'DistanceSNP', u'StrandBias'], dtype='object')
32
64


In [296]:
CPCG0100_X1 = CPCG0100_X1.fillna(0)  #Fill all NA value with 0
print CPCG0100_X1.isnull().sum().sum() 

0


In [314]:
CPCG0100_X2 = pd.concat([CPCG0100_X1, Trinucleotide_onehot, GenomicLocation_onehot],axis=1)
print CPCG0100_X2.isnull().sum().sum() 

0


In [315]:
CPCG0100_X2.columns

Index([u'RefAllele', u'NonRefAllele', u'BaseQual', u'TumourCoverage', u'NormalCoverage', u'MapQual', u'ReadPosition', u'HomopolymerRate', u'GCcontent', u'DistanceSNP', u'StrandBias', u'ACA', u'ACC', u'ACG', u'ACT', u'ATA', u'ATC', u'ATG', u'ATT', u'CCA', u'CCC', u'CCG', u'CCT', u'CNN', u'CTA', u'CTC', u'CTG', u'CTT', u'GCA', u'GCC', u'GCG', u'GCT', u'GTA', u'GTC', u'GTG', u'GTT', u'NCG', u'NTC', u'TCA', u'TCC', u'TCG', u'TCT', u'TTA', u'TTC', u'TTG', u'TTT', u'UTR3', u'UTR5', u'downstream', u'exonic', u'exonic;splicing', u'intergenic', u'intronic', u'ncRNA_UTR3', u'ncRNA_UTR5', u'ncRNA_exonic', u'ncRNA_intronic', u'ncRNA_splicing', u'splicing', u'upstream', u'upstream;downstream'], dtype='object')

In [319]:
CPCG0100_X_Val = CPCG0100_X2.values
CPCG0100_Y_Val = CPCG0100_Y.values

totalSample = CPCG0100_Y_Val.shape[0]
CPCG0100_Y_Val_1 = np.zeros((totalSample,1)) #Will be our label now

print CPCG0100_Y_Val_1.shape

(198526, 1)


In [320]:
CPCG0100_Y_Val[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [321]:
for i in range(totalSample):
    CPCG0100_Y_Val_1[i] = np.argmax(np.bincount(CPCG0100_Y_Val[i]))

In [322]:
CPCG0100_Y_Val_1

array([[ 0.],
       [ 0.],
       [ 0.],
       ..., 
       [ 0.],
       [ 0.],
       [ 0.]])

In [323]:
print np.sum(CPCG0100_Y_Val_1==0)
print np.sum(CPCG0100_Y_Val_1==1)

195673
2853


In [450]:
CPCG0100_X_Val.shape

(198526, 61)

In [324]:
X_train, X_test, Y_train, Y_test = train_test_split(CPCG0100_X_Val, CPCG0100_Y_Val_1, test_size=0.30)

In [325]:
#Logistic Regression
#Logistic Regression-sklearn
logReg = LogisticRegression()
logReg.fit(X_train,Y_train)
Y_logPred      = logReg.predict(X_test)
Y_logPred_prob = logReg.predict_proba(X_test)
print metrics.accuracy_score(Y_test,Y_logPred)

0.985308438833


/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [335]:
logReg.feature

array([[-0.12246952, -0.0625853 ,  0.0046159 , -0.03763786, -0.02480804,
         0.0595277 , -0.02767877, -0.39030191, -0.65942222, -0.00008784,
        -0.60885057, -0.32662922,  0.03639518,  0.60202362, -0.05409527,
        -0.28495187,  0.04207166, -0.05031296,  0.13037274, -0.25478601,
        -0.0428474 ,  0.21792937, -0.08764904,  0.        , -0.08590477,
        -0.11896023, -0.19149371, -0.0220947 , -0.1902036 , -0.07685091,
         0.32170828, -0.07981593, -0.04984001, -0.01804752, -0.30421263,
         0.00766469,  0.        , -0.00000181, -0.27020123,  0.01019143,
         0.23942512, -0.05842536, -0.07107204,  0.01036901, -0.06288853,
         0.04818816, -0.01818775,  0.00286641, -0.03051602, -0.01336008,
        -0.0010206 , -0.2546739 , -0.22972556,  0.00420888, -0.00010265,
        -0.00817632, -0.08873794, -0.00014215, -0.00094149, -0.04571001,
         0.00283173]])

In [336]:

# Feature Importance
from sklearn import datasets
from sklearn import metrics
from sklearn.ensemble import ExtraTreesClassifier
# load the iris datasets
#dataset = datasets.load_iris()
# fit an Extra Trees model to the data
model = ExtraTreesClassifier()
model.fit(X_train,Y_train)
# display the relative importance of each attribute
print(model.feature_importances_)

[ 0.11009872  0.21189289  0.08955475  0.07926225  0.07038555  0.02682453
  0.0649975   0.06801933  0.07087285  0.06703046  0.0668192   0.00208176
  0.00126004  0.00403475  0.00285009  0.00207863  0.00092952  0.0016594
  0.00103226  0.00064033  0.00213579  0.00107276  0.0032725   0.
  0.00147812  0.00176581  0.00143352  0.00109759  0.00137006  0.00121201
  0.00169225  0.0020335   0.00113398  0.00144335  0.0011327   0.00144197
  0.          0.          0.0004568   0.00247828  0.00076111  0.0026218
  0.0018446   0.00184804  0.00130002  0.00153704  0.00085796  0.00038969
  0.00076905  0.00132219  0.00000381  0.00605971  0.00749567  0.00027437
  0.00000042  0.00054387  0.00259491  0.          0.00002102  0.00051116
  0.00026775]


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


In [344]:
CPCG0100_X2.columns

Index([u'RefAllele', u'NonRefAllele', u'BaseQual', u'TumourCoverage', u'NormalCoverage', u'MapQual', u'ReadPosition', u'HomopolymerRate', u'GCcontent', u'DistanceSNP', u'StrandBias', u'ACA', u'ACC', u'ACG', u'ACT', u'ATA', u'ATC', u'ATG', u'ATT', u'CCA', u'CCC', u'CCG', u'CCT', u'CNN', u'CTA', u'CTC', u'CTG', u'CTT', u'GCA', u'GCC', u'GCG', u'GCT', u'GTA', u'GTC', u'GTG', u'GTT', u'NCG', u'NTC', u'TCA', u'TCC', u'TCG', u'TCT', u'TTA', u'TTC', u'TTG', u'TTT', u'UTR3', u'UTR5', u'downstream', u'exonic', u'exonic;splicing', u'intergenic', u'intronic', u'ncRNA_UTR3', u'ncRNA_UTR5', u'ncRNA_exonic', u'ncRNA_intronic', u'ncRNA_splicing', u'splicing', u'upstream', u'upstream;downstream'], dtype='object')

In [163]:
import numpy as np
import pandas as pd, os
import scipy.sparse as sps
import itertools

def one_hot_column(df, cols, vocabs):
    mats = []; df2 = df.drop(cols,axis=1)
    mats.append(sps.lil_matrix(np.array(df2)))
    for i,col in enumerate(cols):
        mat = sps.lil_matrix((len(df), len(vocabs[i])))
        for j,val in enumerate(np.array(df[col])):
            mat[j,vocabs[i][val]] = 1.
        mats.append(mat)

    res = sps.hstack(mats)   
    return res

data = {'state': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
        'year': ['2000', '2001', '2002', '2001', '2002'],
        'pop': [1.5, 1.7, 3.6, 2.4, 2.9]}

df = pd.DataFrame(data)
print df

vocabs = []
vals = ['Ohio','Nevada']
vocabs.append(dict(itertools.izip(vals,range(len(vals)))))
vals = ['2000','2001','2002']
vocabs.append(dict(itertools.izip(vals,range(len(vals)))))

print vocabs

print one_hot_column(df, ['state','year'], vocabs).todense()

   pop   state  year
0  1.5    Ohio  2000
1  1.7    Ohio  2001
2  3.6    Ohio  2002
3  2.4  Nevada  2001
4  2.9  Nevada  2002

[5 rows x 3 columns]
[{'Ohio': 0, 'Nevada': 1}, {'2002': 2, '2000': 0, '2001': 1}]
[[ 1.5  1.   0.   1.   0.   0. ]
 [ 1.7  1.   0.   0.   1.   0. ]
 [ 3.6  1.   0.   0.   0.   1. ]
 [ 2.4  0.   1.   0.   1.   0. ]
 [ 2.9  0.   1.   0.   0.   1. ]]


In [175]:
# -*- coding: utf-8 -*-
""" Small script that shows hot to do one hot encoding
    of categorical columns in a pandas DataFrame.
    See:
    http://scikit-learn.org/dev/modules/generated/sklearn.preprocessing.OneHotEncoder.html#sklearn.preprocessing.OneHotEncoder
    http://scikit-learn.org/dev/modules/generated/sklearn.feature_extraction.DictVectorizer.html
"""
import pandas
import random
import numpy
from sklearn.feature_extraction import DictVectorizer


def one_hot_dataframe(data, cols, replace=False):
    """ Takes a dataframe and a list of columns that need to be encoded.
        Returns a 3-tuple comprising the data, the vectorized data,
        and the fitted vectorizor."""
    vec = DictVectorizer(sparse=False)
    vecData = pd.DataFrame(vec.fit_transform(data[cols].T.to_dict().values()))
    vecData.columns = vec.get_feature_names()
    vecData.index = data.index
    if replace is True:
        data = data.drop(cols, axis=1)
        data = data.join(vecData)
    return (data, vecData, vec)

'''
def one_hot_dataframe(data, cols, replace=False):
    """ Takes a dataframe and a list of columns that need to be encoded.
        Returns a 3-tuple comprising the data, the vectorized data,
        and the fitted vectorizor.
    """
    vec = DictVectorizer()
    mkdict = lambda row: dict((col, row[col]) for col in cols)
    vecData = pandas.DataFrame(vec.fit_transform(data[cols].apply(mkdict, axis=1)).toarray())
    vecData.columns = vec.get_feature_names()
    vecData.index = data.index
    if replace is True:
        data = data.drop(cols, axis=1)
        data = data.join(vecData)
    return (data, vecData, vec)
'''

def main():

    # Get a random DataFrame
    df = pandas.DataFrame(numpy.random.randn(25, 3), columns=['a', 'b', 'c'])

    # Make some random categorical columns
    df['e'] = [random.choice(('Chicago', 'Boston', 'New York', np.nan)) for i in range(df.shape[0])]
    df['f'] = [random.choice(('Chrome', 'Firefox', 'Opera', "Safari")) for i in range(df.shape[0])]
    print df

    # Vectorize the categorical columns: e & f
    df, _, _ = one_hot_dataframe(df, ['e', 'f'], replace=True)
    print df

#if __name__ == '__main__':
main()

           a         b         c         e        f
0   0.173710 -2.915045  0.008241       NaN    Opera
1  -1.034360 -1.733141 -1.587523       NaN   Chrome
2  -1.463398 -0.206447 -1.248982    Boston    Opera
3   0.335188  0.516611  1.143118   Chicago   Chrome
4   0.372292 -1.297359  1.474775  New York    Opera
5   0.351997  0.216036  0.712386       NaN    Opera
6  -1.115179  0.617589 -0.388718    Boston   Safari
7   1.257660 -1.370415 -0.341618    Boston   Safari
8  -2.044490 -0.032497  1.080068       NaN  Firefox
9  -0.583843  0.204096 -0.079827    Boston   Safari
10 -1.225954 -0.774935 -0.896044       NaN    Opera
11  2.037362 -0.104686  0.842412  New York    Opera
12 -0.939473 -0.176803  0.717135   Chicago  Firefox
13 -0.949167  0.112423 -0.088719       NaN  Firefox
14  1.847247 -0.761243 -0.243570       NaN   Safari
15 -0.013457 -0.282127 -0.921607  New York   Chrome
16 -1.160175  0.363111  3.259665       NaN   Chrome
17 -0.198332 -1.143464 -0.178219   Chicago   Safari
18 -0.529146

In [445]:
print type(CPCG0100_X_Val)
print type(CPCG0100_Y_Val_1)

<type 'numpy.ndarray'>
<type 'numpy.ndarray'>


In [446]:
type(CPCG0100_X)
print CPCG0100_Y_Val_1

[[ 0.]
 [ 0.]
 [ 0.]
 ..., 
 [ 0.]
 [ 0.]
 [ 0.]]


In [352]:
CPCG0100_dataframe = pd.concat([CPCG0100_X, pd.DataFrame(CPCG0100_Y_Val_1)],axis=1)

In [353]:
CPCG0100_dataframe.shape

(198526, 14)

In [357]:
CPCG0100_dataframe.to_csv('CPCG0100.csv')

In [454]:
####################################other csv#################
CPCG0100_Val_dataframe = pd.concat([pd.DataFrame(CPCG0100_X_Val), pd.DataFrame(CPCG0100_Y_Val_1)],axis=1)

In [461]:
CPCG0100_X_Val

array([[  183.,     3.,  4490., ...,     0.,     0.,     0.],
       [  177.,    10.,  4691., ...,     0.,     0.,     0.],
       [  115.,    10.,  3512., ...,     0.,     0.,     0.],
       ..., 
       [  127.,    44.,  5099., ...,     0.,     0.,     0.],
       [  276.,    59.,  9749., ...,     0.,     0.,     0.],
       [   38.,    20.,  1689., ...,     0.,     0.,     0.]])

In [457]:
CPCG0100_Val_dataframe.to_csv('CPCG0100_Val.csv')

In [458]:
CPCG0100_Val_dataframe.shape

(198526, 62)

In [364]:
### R code start here ####

In [365]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [370]:
%%R
# Read in the data
CPCG0100Frame = read.csv("CPCG0100.csv")
str(CPCG0100Frame)

'data.frame':	198526 obs. of  15 variables:
 $ X              : int  0 1 2 3 4 5 6 7 8 9 ...
 $ RefAllele      : int  183 177 115 83 96 120 117 92 27 9 ...
 $ NonRefAllele   : int  3 10 10 26 20 11 14 8 3 3 ...
 $ BaseQual       : int  4490 4691 3512 3251 3255 3672 3865 2946 524 342 ...
 $ TumourCoverage : int  231 229 137 119 127 155 155 123 36 17 ...
 $ NormalCoverage : int  133 134 74 59 53 108 108 75 26 15 ...
 $ MapQual        : num  25 25 0 37 29 0 0 0 27 0 ...
 $ ReadPosition   : num  55 60 52 38 47 42 51 55 70.5 64 ...
 $ Trinucleotide  : Factor w/ 35 levels "ACA","ACC","ACG",..: 32 14 28 18 19 24 34 18 35 6 ...
 $ HomopolymerRate: num  4.7 4.53 2.71 2.75 2.72 ...
 $ GCcontent      : num  0.587 0.587 0.532 0.677 0.677 ...
 $ GenomicLocation: Factor w/ 16 levels "","downstream",..: 1 1 1 8 1 1 8 1 1 5 ...
 $ DistanceSNP    : int  41830 41825 35658 34719 34700 32011 31988 23727 23645 10933 ...
 $ StrandBias     : num  0.368 0.371 0.336 0.462 0.409 ...
 $ X0             : num  0 0

In [419]:
%%R
sum(is.na(CPCG0100Frame$StrandBias))   #ReadPosition=32, StrandBias=32

#Imputing na value
#df = data.frame(x = 1:20, y = c(1:10,rep(NA,10)))
#df$y[is.na(df$y)] = mean(df$y, na.rm=TRUE)
# alternative
#df = transform(df, y = ifelse(is.na(y), mean(y, na.rm=TRUE), y))

CPCG0100Frame$ReadPosition[is.na(CPCG0100Frame$ReadPosition)] = mean(CPCG0100Frame$ReadPosition, na.rm=TRUE)
CPCG0100Frame$StrandBias[is.na(CPCG0100Frame$StrandBias)] = mean(CPCG0100Frame$StrandBias, na.rm=TRUE)

print (sum(is.na(CPCG0100Frame$ReadPosition)) )
print (sum(is.na(CPCG0100Frame$StrandBias)) )

[1] 0
[1] 0


In [420]:
%%R
CPCG0100Frame$Trinucleotide  = as.factor(CPCG0100Frame$Trinucleotide)
CPCG0100Frame$GenomicLocation= as.factor(CPCG0100Frame$GenomicLocation)

In [431]:
%%R
library(caTools)
split = sample.split(CPCG0100Frame$X0, SplitRatio = 0.80) #60% train, 40% test
train_R = subset(CPCG0100Frame, split==TRUE)
test_R = subset(CPCG0100Frame, split==FALSE)

In [432]:
%%R
#install.packages("caTools")
str(CPCG0100Frame)

'data.frame':	198526 obs. of  15 variables:
 $ X              : int  0 1 2 3 4 5 6 7 8 9 ...
 $ RefAllele      : int  183 177 115 83 96 120 117 92 27 9 ...
 $ NonRefAllele   : int  3 10 10 26 20 11 14 8 3 3 ...
 $ BaseQual       : int  4490 4691 3512 3251 3255 3672 3865 2946 524 342 ...
 $ TumourCoverage : int  231 229 137 119 127 155 155 123 36 17 ...
 $ NormalCoverage : int  133 134 74 59 53 108 108 75 26 15 ...
 $ MapQual        : num  25 25 0 37 29 0 0 0 27 0 ...
 $ ReadPosition   : num  55 60 52 38 47 42 51 55 70.5 64 ...
 $ Trinucleotide  : Factor w/ 35 levels "ACA","ACC","ACG",..: 32 14 28 18 19 24 34 18 35 6 ...
 $ HomopolymerRate: num  4.7 4.53 2.71 2.75 2.72 ...
 $ GCcontent      : num  0.587 0.587 0.532 0.677 0.677 ...
 $ GenomicLocation: Factor w/ 16 levels "","downstream",..: 1 1 1 8 1 1 8 1 1 5 ...
 $ DistanceSNP    : int  41830 41825 35658 34719 34700 32011 31988 23727 23645 10933 ...
 $ StrandBias     : num  0.368 0.371 0.336 0.462 0.409 ...
 $ X0             : num  0 0

In [434]:
%%R
#79410+119116
CPCG0100Log = glm(X0 ~ ., data = CPCG0100Frame, family=binomial)
summary(CPCG0100Log)


Call:
glm(formula = X0 ~ ., family = binomial, data = CPCG0100Frame)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-8.4904  -0.1760  -0.0841  -0.0134   8.4904  

Coefficients:
                                     Estimate Std. Error z value Pr(>|z|)    
(Intercept)                        -7.527e+00  3.224e-01 -23.343  < 2e-16 ***
X                                  -2.117e-06  3.377e-07  -6.269 3.62e-10 ***
RefAllele                          -1.213e-01  5.536e-03 -21.917  < 2e-16 ***
NonRefAllele                       -6.052e-02  5.290e-03 -11.441  < 2e-16 ***
BaseQual                            4.522e-03  1.692e-04  26.727  < 2e-16 ***
TumourCoverage                     -3.752e-02  2.775e-03 -13.519  < 2e-16 ***
NormalCoverage                     -2.038e-02  1.503e-03 -13.558  < 2e-16 ***
MapQual                             1.284e-01  4.074e-03  31.508  < 2e-16 ***
ReadPosition                        3.865e-03  2.066e-03   1.871 0.061351 .  
TrinucleotideACC     

In [435]:
%%R
#confint(CPCG0100Log)
#str(CPCG0100Log)
predictCPCG0100 = predict(CPCG0100Log, type="response") #, newdata=CPCG0100Frame)

In [442]:
%%R
#predictTest = predict(CPCG0100Log, type="response", newdata=test_R)
table(CPCG0100Frame$X0, predictCPCG0100 > 0.5)

   
     FALSE   TRUE
  0 195527    146
  1   2835     18


In [444]:
(195527+18)/(195527+18+146+2835.0)

0.9849843345456011